In [251]:
from selenium import webdriver  # 동적크롤링
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time
from tqdm import tqdm

In [252]:
def review_scraping(driver, query, review_list, review_cnt):
    reviews = [] # 각 업체별 리뷰 리스트
    review__count = 0
    url = f"https://map.naver.com/p/search/{query}" 
    driver.get(url)
    time.sleep(3)
    
    try:
        # iframe 전환
        iframe_element = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.ID, "entryIframe"))
            )

        driver.switch_to.frame(iframe_element)
        time.sleep(2)

        try:
            # 특정 <a> 태그 찾기 - '방문자 리뷰' 링크
            tag = driver.find_element(By.XPATH, '//a[@role="button" and contains(@href, "/review/visitor")]')
            # 태그에서 href 속성 값 추출
            href = tag.get_attribute('href')
            # 추출한 URL로 이동 (네이버 플레이스 - 리뷰로 이동)
            driver.get(href)
            time.sleep(3) #3초 대기
            
            # 리뷰 수 추출
            try:
                # <em> 태그에서 전체 리뷰 수 추출(str -> int): 반복문에서 사용
                review_counts = int(driver.find_element(By.CSS_SELECTOR, 'em.place_section_count').text.replace(',', ''))
                review__count = review_counts
                print('총 후기수 = ', review_counts)
                if review_counts > 100:
                    review_count = 100
                else:
                    review_count = review_counts
                print('사용 후기수 = ', review_count)
                
            except:
                review__count = 0
                print("no_review_count")
            
            review_cnt.append(review__count)
            
            # 스크롤 맨밑으로 내리기
            # try:
            #     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 스크롤 맨밑으로 내리기 (그래야 div:nth-child(6) 이부분이 성립됨,,,하..)
            #     time.sleep(2)
            # except:
            #     print("scroll_fail")
            
            # 더보기 버튼
            try:
                # By.CSS_SELECTOR -> 더보기버튼 요소 찾기 (최대 10초동안 찾기)
                nxt_button_element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div:nth-child(3) > div.place_section.k1QQ5 > div.NSTUp > div > a'))
                        )
                print("nxt_button_find_success") 
                
                # 더보기 버튼 클릭 반복 -> 리뷰 전부 펼치기
                click_count = 0
                while click_count < 12:
                    try:
                        nxt_button_element.click()
                        time.sleep(1)  # 페이지 로드 대기 시간
                        nxt_button_element = WebDriverWait(driver, 10).until(
                                    EC.presence_of_element_located((By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div:nth-child(3) > div.place_section.k1QQ5 > div.NSTUp > div > a'))
                                )
                        click_count += 1
                    except Exception as e:
                        print(f'더보기 버튼 클릭 실패: {e}')
                        break
                print("전체 더보기 완료")
                
            except Exception as e:
                print(f"nxt_button_find_fail: {e}")
                time.sleep(1)
            
            try:
                try:
                    # By.CSS_SELECTOR -> ul 요소 찾기
                    review_list_element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div:nth-child(3) > div.place_section.k1QQ5 > div > ul'))
                        )
                    print("ul_success")

                    time.sleep(2)
                except:
                    try:
                        # By.CSS_SELECTOR -> ul 요소 찾기
                        review_list_element = WebDriverWait(driver, 10).until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.k1QQ5 > div > ul'))
                            )
                        print("ul_success")

                        time.sleep(2)
                    except:
                        print('not find ul')
                
                # 리뷰 추출
                for i in range(1, review_count+1): # 1부터 시작
                    try: 
                        # By.CSS_SELECTOR -> div 요소 찾기
                        review_div = review_list_element.find_element(By.CSS_SELECTOR, f'li:nth-child({i}) > div > div.vg7Fp.CyA_N')
                        review = review_div.find_element(By.XPATH, './a/span[1]').text
                        time.sleep(2)
                        reviews.append(review)
                    except:
                        try:
                            # By.CSS_SELECTOR -> div 요소 찾기
                            review_div = review_list_element.find_element(By.CSS_SELECTOR, f'li:nth-child({i}) > div > div.vg7Fp')
                            review = review_div.find_element(By.XPATH, './a/span[1]').text
                            time.sleep(2)
                            reviews.append(review)
                        except:      
                            print(f'Review {i} not found')
                            reviews.append('텍스트없음')
            
            except:
                review_list.append('리뷰없음')
                print("Review_find_fail")
        except:
            review_list.append('방문자 리뷰 없음')
            review_cnt.append(0)
            print("방문자 리뷰 없음")     
    except:
        review_list.append('검색불가')
        review_cnt.append(0)
        print('search fail!!')
        
    finally:
        #리뷰 저장
        if reviews:
            review_list.append(reviews)
                
        print(f'{query} - 리뷰 저장 완료')

데이터 불러오기

In [274]:
import pandas as pd
data = pd.read_csv('C:/Users/User/big_project/문욱예원0_1566_마저하자3.csv')

!!!!!!!!!!!!!!! 인덱스 번호 참고하세요 !!!!!!!!!!!!!!!!!!!!

!!! 지도에 안뜨거나 / 도로명주소 또는 지번주소가 다름(이사 추정) => 인덱스 :
1, 2, 9, 11 ,13, 14, 26, 35, 43, 56, 63, 88, 97, 120, 167, 169, 170, 173, 179, 183, 189, 199, 211, 234, 238, 242, 243, 249, 268, 275, 290, 300, 306, 355, 366, 377, 388, 408, 418, 450, 458,473, 487, 492, 504, 542, 591, 604, 674, 679, 736, 737, 887, 894, 897, 963, 1436, 1437, 1450

!!! 네이버지도에 쿼리 입력시 두개가 떠서 작동이 안되는 인덱스 : 
797, 1005, 1020, 1058, 1121, 1128, 1139, 1209, 1233, 1266, 1276, 1327, 1335, 1358, 1444, 1457, 1470, 1490, 1491, 1532, 1533, 1556

In [244]:
# 각자 해당하는 범위 입력
# query = data['naver_query'].iloc[91:92]

In [245]:
# query

91    서울특별시 구로구 구로중앙로28길 24 부뚜막청국장
Name: naver_query, dtype: object

In [86]:
# data = pd.read_excel("C:/Users/User/big_project/preprocessed_okay_kids.xlsx")
# data['query'] = data['새주소[도로명 주소]'] + ' ' + data['콘텐츠명']
# query = data['query'].iloc[9:30]

In [275]:
query_list_1 = [80,139,145,150,196,202,217,228,230,239,253,265,272,292,294,310,340,386,387,392,427,435,439,461,479,481,482,495,500,501,518,538,539,544,555,569,578,586,588,597,
                598,600,614,633,635,638,646,659,687,694,705,712,717,721,765,775,818,851,853,880,888,906,921,926,929,937,938,944,945,954,955,960,973,990,997,1000,1001,1027,1028,
                1034,1068,1111,1129,1177]

query_list_2 = [1204,1206,1226,1233,1240,1247,1259,1276,1283,1296,1314,1319,1325,1327,1345,1350,1398,1406,1432,1451,1456,1460,1500,1522,1523,1527,1538,1557]

In [ ]:
driver = webdriver.Chrome()
naver_rev = []
naver_cnt = []
for i in tqdm(query_list_2, desc="Processing queries", ncols=100):
    tqdm.write(f'{i}번째 시작')
    q = data['naver_query'][i]
    review_scraping(driver, q, naver_rev, naver_cnt)

각 행에 데이터 넣기

In [277]:
# for i in range(len(naver_rev)):
#     data.at[i+91, 'naver_review'] = naver_rev[i]
#     data.at[i+91, 'naver_review_cnt'] = naver_cnt[i]
    
for i, j in enumerate(query_list_2):
    data.at[j, 'naver_review'] = naver_rev[i]
    data.at[j, 'naver_review_cnt'] = naver_cnt[i]

In [279]:
data.to_csv('문욱예원0_1566_마저하자4.csv', index=False)

In [225]:
data2 = pd.read_csv('C:/Users/User/big_project/문욱0_400_수정2.csv')

In [241]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1566 entries, 0 to 1565
Data columns (total 34 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   콘텐츠명               1566 non-null   object 
 1   구명                 1566 non-null   object 
 2   새주소[도로명 주소]        1522 non-null   object 
 3   지번주소               857 non-null    object 
 4   좌표[X]              1566 non-null   float64
 5   좌표[Y]              1566 non-null   float64
 6   전화번호               1433 non-null   object 
 7   주요시설               28 non-null     object 
 8   중분류                1566 non-null   object 
 9   대분류                1566 non-null   object 
 10  실내_실외              1566 non-null   object 
 11  시군                 602 non-null    object 
 12  키워드                596 non-null    object 
 13  웹URL               926 non-null    object 
 14  키즈메뉴(아이가먹을수있는 메뉴)  569 non-null    object 
 15  유아용품               570 non-null    object 
 16  영업시간               568 n

In [240]:
data2['naver_review'].iloc[391]

'[\'신림 찐맛집~! 워낙 자주 방문하는 곳이기도 해요~~ 서울에 이렇게 맛있는 막국수집이 있다는 것만으로도 행복해요~ 주문하는 메뉴마다 다 맛있어요~~ 동치미막국수, 비빔막국수, 판메밀 모두모두 자가제면으로 만들어 더욱 맛있어요~! 제대로된 막국수 드시고 싶으면 막불감동이네요~!\', \'새우교자, 비빔, 판메밀 모두 맛있어요! 동치미 막국수 못먹은게 아쉬워 밀키트 사왔는데 깔끔하고 직화 불고기부터 열무김치까지 다 들어있어서 집에서 먹어도 맛있었어요! 라! 비빔은 매운맛으로 시켰더니 칼칼해서 더 맛있었답니다😋\', \'신림가성비맛집 막불감동 엄마가 메밀칼국수를 너무 좋아해 자주 방문하는데 시원한 동치미막국수가 땡겨 남편이랑도 와봤어요! 역시나 맛있는 막불감\\n동메뉴 모든 메뉴 하나하나에 정성가득이 느껴지고 재료하나에도 신선함과 양념소스도 막불감동만의 특별함이 있어요 신림찐맛집 막불감동은 한번 방\\n문하게되면 단골될수밖에 없네요 ㅎㅎ 매장도 쾌적하고 직원분들도 친절해 올때마다 함박웃음이 지어져요 ㅎㅎ 담에 또 방문할게요\', \'친한 동생들이랑 방문했어요 동치미 냉메밀 짬뽕칼국수 새우지짐만두등 다양하게 먹었는데 가격이 굉장히 저렴해서 부담없고 좋았습니다 세트엔 숯불\\n고기까지 나와주니 엄청 든든하더라구요 깔끔하고 건강한맛! 근데 정말 맛있어요 매일 와도 질리지 않을것 같네요 ㅎㅎ\', \'남편이랑 친정엄마 모시구 저녁먹으러 방문했었어요:) 주문한 메뉴 전부 간이 너무 쎄지않고 슴슴하면서 깔끔한 맛이라 부담없이 저녁식사하기 좋았습\\n니다! 대표메뉴인 막국수 외에 짬뽕도 제대로 진국이었고요. 만두는 진짜 꼭 필수로 먹어야해요!! 배부른데도 끝까지 다 먹게 한 식당이었어요><\', \'등산갔다 내려와서 먹는 맛집으로 막불감동이 기사에 나왔음. 대모산 등산 갔다 내려오며 남편이 막불감동으로 나오라고 연락왔음. 나는 등산을 안했지\\n만 막국수 먹으러 달려감 ㅋ 시원한 동치미 막국수 세트랑 메밀 새우교자 먹었음. 언제 먹어도 맛있는 막국수 맛집 인정!\', \

In [104]:
# 문자열 처리를 통해 개별 리뷰 추출
reviews_str = data2['naver_review'][195]  # 데이터프레임에서 첫 번째 리뷰 문자열 가져오기
reviews_list = eval(reviews_str)  # 문자열을 리스트로 변환

print(len(reviews_list))

# # 클린 리뷰 리스트 생성
# cleaned_reviews = [input_process_text(review) for review in reviews_list if review.strip()]

# # 결과 출력
# for idx, review in enumerate(cleaned_reviews, start=1):
#     print(f'Cleaned Review {idx}: {review}')

# cleaned_list = list(filter(lambda x: x != '', cleaned_reviews)) # 빈텍스트 처리
# # 텍스트가 없이 느낌표, 점, 물음표만 있는 항목 제거
# cleaned_list = [item for item in cleaned_list if not all(char in ['!', '.', '?'] for char in item)] # 문자로만 이루어져있는 리뷰들 처리

100
